In [1]:
from datasets import create_dataloader
from general import colorstr, check_img_size
import numpy as np

In [2]:
import yaml

In [3]:
train_path = "../coco_kpts/train2017.txt"  # 118287 images
val_path = "../coco_kpts/val2017.txt"  # 5000 images"

# number of classes
nc = 1

names = ['person']

gs = 32 # grid size
img_size = [640, 640] # [train, test]
imgsz, imgsz_test = [check_img_size(x, gs) for x in img_size]

batch_size = 4


hyp_file = "hyp.yaml"
with open(hyp_file) as f:
    hyp = yaml.safe_load(f)  # load hyps

cache_images=False
rank=-1
world_size=1
workers=4,
image_weights=False 
quad=False
kpt_label = True

In [4]:
img_size.extend([img_size[-1]] * (2 - len(img_size)))
print(img_size)

[640, 640]


In [5]:
dataloader, dataset = create_dataloader(train_path, imgsz, batch_size, gs,
                                            hyp=hyp, augment=True, cache=cache_images, rank=rank,
                                            world_size=world_size, workers=workers,
                                            image_weights=image_weights, quad=quad, prefix=colorstr('train: '), kpt_label=kpt_label)

train: Scanning '..\coco_kpts\train2017.cache' images and labels... 56599 found, 0 missing, 0 empty, 0 corrupted: 100%|


In [6]:
mlc = np.concatenate(dataset.labels, 0)[:, 0].max()  # max label class
nb = len(dataloader)  # number of batches

In [7]:
print(mlc, nb)

0.0 14150


In [8]:
pbar = enumerate(dataloader)

In [10]:
epoch = 1
for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
    #print(paths)
    if i>10:
        break
    ni = i + nb * epoch  # number integrated batches (since train start)
    imgs = imgs.to(device, non_blocking=True).float() / 255.0  # uint8 to float32, 0-255 to 0.0-1.0

    # Warmup
    if ni <= nw:
        xi = [0, nw]  # x interp
        # model.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
        accumulate = max(1, np.interp(ni, xi, [1, nbs / total_batch_size]).round())
        for j, x in enumerate(optimizer.param_groups):
            # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
            x['lr'] = np.interp(ni, xi, [hyp['warmup_bias_lr'] if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
            if 'momentum' in x:
                x['momentum'] = np.interp(ni, xi, [hyp['warmup_momentum'], hyp['momentum']])

    # Multi-scale
    if opt.multi_scale:
        sz = random.randrange(imgsz * 0.5, imgsz * 1.5 + gs) // gs * gs  # size
        sf = sz / max(imgs.shape[2:])  # scale factor
        if sf != 1:
            ns = [math.ceil(x * sf / gs) * gs for x in imgs.shape[2:]]  # new shape (stretched to gs-multiple)
            imgs = F.interpolate(imgs, size=ns, mode='bilinear', align_corners=False)

    # Forward
    with amp.autocast(enabled=cuda):
        pred = model(imgs)  # forward
        loss, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size
        if rank != -1:
            loss *= opt.world_size  # gradient averaged between devices in DDP mode
        if opt.quad:
            loss *= 4.

    # Backward
    scaler.scale(loss).backward()

    # Optimize
    if ni % accumulate == 0:
        scaler.step(optimizer)  # optimizer.step
        scaler.update()
        optimizer.zero_grad()
        if ema:
            ema.update(model)

    # Print
    if rank in [-1, 0]:
        mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
        mem = '%.3gG' % (torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0)  # (GB)
        s = ('%10s' * 2 + '%10.4g' * 8) % (
            '%g/%g' % (epoch, epochs - 1), mem, *mloss, targets.shape[0], imgs.shape[-1])
        pbar.set_description(s)

        # Plot
        if plots and ni < 33:
            f = save_dir / f'train_batch{ni}.jpg'  # filename
            plot_images(imgs, targets, paths, f, kpt_label=kpt_label)
            #Thread(target=plot_images, args=(imgs, targets, paths, f), daemon=True).start()
            # if tb_writer:
            #     tb_writer.add_image(f, result, dataformats='HWC', global_step=epoch)
            #     tb_writer.add_graph(torch.jit.trace(model, imgs, strict=False), [])  # add model graph
        elif plots and ni == 10 and wandb_logger.wandb:
            wandb_logger.log({"Mosaics": [wandb_logger.wandb.Image(str(x), caption=x.name) for x in
                                          save_dir.glob('train*.jpg') if x.exists()]})

NameError: name 'device' is not defined